<a href="https://colab.research.google.com/github/umbertogagl97/Tesi/blob/main/Impronte/AddestramentoScanner/Addestramento_scanner_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Init**

## Import librerie

In [ ]:
import tensorflow as tf
import numpy as np

##Def variabili

In [ ]:
#Scanner
scanner_name = 'DigitalPersona'

#salvataggio modello
model_save_name = 'VGG19_first_trial'
path_model_save = F"/content/gdrive/My Drive/ModelliCNN/Scanner/{scanner_name}/{model_save_name}" 

#load dataset training
pathDataset=F'/content/gdrive/MyDrive/Dataset_impronte/training/{scanner_name}'

##Parametri

In [ ]:
batch_size=5
num_class=2

##Collegamento google drive

In [ ]:
#collegamento google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#**Dataset**

##Caricamento dataset

In [ ]:
train_ds= tf.keras.utils.image_dataset_from_directory(
    pathDataset,
    label_mode='categorical',
    image_size=(224,224)) #effettua il resize delle immagini

Found 2000 files belonging to 2 classes.


##Calcolo dimensioni dataset

In [ ]:
import os
list = os.listdir(pathDataset) # dir is your directory path
dim_set=0
for x in list:
  dim_set=dim_set+(len(os.listdir(os.path.join(pathDataset,x))))
print(dim_set)

2000


##Calcolo dimensioni immagini

In [ ]:
image_batch, labels_batch = next(iter(train_ds))
im_shape=image_batch[0].shape
del image_batch,labels_batch

##Creazione matrici

In [ ]:
x_train = np.zeros((dim_set, im_shape[0], im_shape[1], im_shape[2]), dtype=np.uint8)
y_train = np.zeros((dim_set,num_class), dtype=np.uint8)

In [ ]:
x_train.shape

(2000, 224, 224, 3)

In [ ]:
i=0
j=0
for image_batch, labels_batch in train_ds:
  j=i+image_batch.get_shape()[0]
  #print("i: "+str(i))
  #print("j: "+str(j))
  x_train[i : j,:,:,:]=image_batch
  y_train[i : j ,:]=labels_batch
  i=j

del i,j  

##Preprocessing immagini

In [ ]:
min_, max_ = np.amin(x_train), np.amax(x_train)
x_train = (x_train - min_) / (max_ - min_)

#**Creazione modello**

###Load model pre-trained

In [ ]:
#creazione rete usando vgg16 preaddestrata e aggiungendo gli ultimi livelli per adattarla al problema di 10 classi
from tensorflow.keras import applications as applications

#importa rete vgg16 addestrata sul dataset imagenet, esclude gli ultimi livelli e come input pongo dimensioni 96,96,3 
model_conv = applications.VGG19(weights='imagenet', include_top=False, input_shape=(im_shape[0],im_shape[1],im_shape[2]))

# Freeze all the layers (non modifico i pesi dei livelli inferiori)
for layer in model_conv.layers[:]:
    layer.trainable = False

model_conv.summary()    

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

#**Training**
> per riaddestrare eseguire solo questa sezione



##Aggiunta classificatore

In [ ]:
from tensorflow.keras import layers,models,losses,optimizers,applications
import numpy as np

# creo un modello aggiungendo livelli alla rete importata
model = models.Sequential()
# Add the vgg convolutional base model
model.add(model_conv)
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dense(num_class, activation='softmax')) #10: numero di valori in uscita (le classi)

#stampa info del modello
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 7, 7, 512)         20024384  
_________________________________________________________________
flatten_5 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_12 (Dense)             (None, 4096)              102764544 
_________________________________________________________________
dense_13 (Dense)             (None, 4096)              16781312  
_________________________________________________________________
dense_14 (Dense)             (None, 2)                 8194      
Total params: 139,578,434
Trainable params: 119,554,050
Non-trainable params: 20,024,384
_________________________________________________________________


##Configura parametri modello

In [ ]:
# Configure the model for training (setta i parametri del modello)
model.compile(
        loss=losses.categorical_crossentropy, optimizer=optimizers.SGD(learning_rate=1e-6,momentum=0.9), metrics=["accuracy"]
    )

##Addestramento

In [ ]:
history= model.fit(x_train,y_train,epochs=40,batch_size=5,validation_split=0.2)

Epoch 1/40
320/320 [==============================] - 64s 198ms/step - loss: 0.6865 - accuracy: 0.5525 - val_loss: 0.6649 - val_accuracy: 0.6325
Epoch 2/40
320/320 [==============================] - 73s 227ms/step - loss: 0.6389 - accuracy: 0.7106 - val_loss: 0.6285 - val_accuracy: 0.7000
Epoch 3/40
320/320 [==============================] - 63s 196ms/step - loss: 0.6015 - accuracy: 0.7919 - val_loss: 0.5918 - val_accuracy: 0.7850
Epoch 4/40
320/320 [==============================] - 73s 227ms/step - loss: 0.5700 - accuracy: 0.8244 - val_loss: 0.5615 - val_accuracy: 0.8350
Epoch 5/40
320/320 [==============================] - 63s 197ms/step - loss: 0.5430 - accuracy: 0.8487 - val_loss: 0.5387 - val_accuracy: 0.8350
Epoch 6/40
320/320 [==============================] - 63s 197ms/step - loss: 0.5194 - accuracy: 0.8581 - val_loss: 0.5210 - val_accuracy: 0.8175
Epoch 7/40
320/320 [==============================] - 73s 227ms/step - loss: 0.5003 - accuracy: 0.8600 - val_loss: 0.4982 - val_ac

KeyboardInterrupt: ignored

#**Salvataggio modello**

In [ ]:
#salva modello su drive
model.save(path_model_save)

INFO:tensorflow:Assets written to: /content/gdrive/My Drive/ModelliCNN/Scanner/DigitalPersona/VGG19_first_trial/assets
